In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [2]:
ratings_matrix = pd.read_pickle('ratings_matrix.pkl')
#ratings_with_name = pd.read_pickle('ratings_with_name.pkl')

In [3]:
corr_matrix = ratings_matrix.corr(method='pearson', min_periods=30)

In [4]:
user_ratings= []
for i in range(0,len(ratings_matrix)):
    user_ratings.append(ratings_matrix.loc[i].dropna())

In [14]:
user_ratings[0]

Movie:Pretty Woman                         4.0
Movie:P.S I Love You                       4.0
Movie:Memento                              1.0
Movie:L.A Confidential                     3.0
Movie:Taken                                5.0
Movie:Forrest Gump                         3.0
Movie:Indiana Jones                        4.0
Movie:Avatar                               5.0
Movie:Gone Girl                            5.0
Movie:Da Vinci Code                        3.0
Book:Girl on the Train                     2.0
Book:Gone Girl].                           4.0
Book:Girl with the Dragon tattoo           4.0
Book:Kane and Abel                         3.0
Book:Goosebumps                            3.0
Book:Treasure Island                       2.0
Book:Moby Dick                             4.0
Book:Harry Potter : Philosopher’s Stone    4.0
Name: 0, dtype: float64

In [15]:
def generate_similar_items_list(corr_mat,user_rat):
    final_recommendations = []
    
    for i in range(0,len(user_rat)):
        recommendations = pd.Series()
        for j in range(0,len(user_rat[i].index)):
            rated_items = user_rat[i].index[j]
            similar_items = corr_mat[rated_items].dropna()
            similar_items = similar_items.map(lambda x : x * user_rat[i][j])
            recommendations = recommendations.append(similar_items)
        recommendations = recommendations.groupby(recommendations.index).sum()
        final_recommendations.append(recommendations)
    
    return final_recommendations

In [16]:
recommendations = generate_similar_items_list(corr_mat=corr_matrix, user_rat= user_ratings)

In [21]:
recommendations[14].sort_values(ascending=False) #recommendations for the 14th index user (HIGHER VALUE IMPLIES HIGHER RATING FOR THE PARTICULAR USER)

Book:Da vinci Code                             55.540689
Movie:Sherlock(Series)                         54.427771
Movie:24(Series)                               54.164633
Book:The Shining                               54.039158
Movie:The Girl with the Dragon Tattoo          53.393771
Movie:Prison Break(Series)                     52.665907
Book:Around The World in 80 Days               52.628653
Book:Hounds Of Baskerville(Sherlock Holmes)    52.309965
Book:Hardy Boys                                52.268291
Movie:Taken                                    51.943786
Movie:Forrest Gump                             51.585384
Movie:Doctor Who(Series)                       50.070370
Book:Murder on the orient express              49.901155
Book:Famous Five                               49.842021
Movie:Gone Girl                                49.668400
Book:Gone Girl].                               49.179360
Movie:Avatar                                   48.207454
Book:Pride and Prejudice       

In [29]:
def scaled_ratings(recommendations):
    scaled_recommendations = []
    for i in range(0,len(recommendations)):
        rec = recommendations[i].sort_values(ascending = False)
        rec = (rec - rec.min())/(rec.max() - rec.min()) #normalize it to 0-1
        rec = rec*4 + 1  #scale it for 1-5 rating range
        rec = np.rint(rec) #get it to the nearest integer
        scaled_recommendations.append(rec)
    return scaled_recommendations    

In [30]:
scaled_recommendations =  scaled_ratings(recommendations)

In [31]:
scaled_recommendations[14] #scales everything to a range 1-5 so that we can calculate error in a realistic scenario

Book:Da vinci Code                             5.0
Movie:Sherlock(Series)                         5.0
Movie:24(Series)                               5.0
Book:The Shining                               5.0
Movie:The Girl with the Dragon Tattoo          5.0
Movie:Prison Break(Series)                     5.0
Book:Around The World in 80 Days               5.0
Book:Hounds Of Baskerville(Sherlock Holmes)    5.0
Book:Hardy Boys                                5.0
Movie:Taken                                    5.0
Movie:Forrest Gump                             5.0
Movie:Doctor Who(Series)                       5.0
Book:Murder on the orient express              5.0
Book:Famous Five                               5.0
Movie:Gone Girl                                5.0
Book:Gone Girl].                               5.0
Movie:Avatar                                   4.0
Book:Pride and Prejudice                       4.0
Movie:Pretty Woman                             4.0
Book:The Kite Runner           

In [32]:
def error_calculation(predicted_ratings,orig_ratings_mat):
    """
    predicted_ratings and orig_ratings_mat should be of same length
    
    """
    error_array = []
    for i in range(0,len(predicted_ratings)):
        common_items = predicted_ratings[i].index.intersection(orig_ratings_mat.loc[i].dropna().index)
        pred = predicted_ratings[i][common_items]
        orig = orig_ratings_mat.loc[i][common_items]
        pred  = np.array(pred)
        orig  = np.array(orig)
        error_score = mean_absolute_error(orig,pred)
        error_array.append(error_score)
        print(i) #to debug in case a paricular row has some value which can cause error
    return error_array

In [33]:
errors = error_calculation(scaled_recommendations,ratings_matrix)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [34]:
errors = np.array(errors)

In [35]:
errors.mean()

0.8411153557831221

In [39]:
predicted_for_user = scaled_recommendations[14]
actual_user_ratings = user_ratings[14]

In [42]:
common = predicted_for_user.index.intersection(actual_user_ratings.index)

In [43]:
predicted_for_user = predicted_for_user[common]

In [64]:
compare_df  = pd.concat([predicted_for_user,actual_user_ratings],axis = 1,sort = True)

In [66]:
compare_df.rename(index = str , columns={0 : "Predicted" , 14 : "Actual"})

,Predicted,Actual
Book:1984,3.0,4.0
Book:Alice in the Wonderland,4.0,4.0
Book:Around The World in 80 Days,5.0,3.0
Book:Da vinci Code,5.0,5.0
Book:Death on the Nile,4.0,4.0
Book:Famous Five,5.0,4.0
Book:Goosebumps,3.0,2.0
Book:Hardy Boys,5.0,4.0
Book:Harry Potter : Half Blood Prince,4.0,5.0
Book:Harry Potter : Philosopher’s Stone,4.0,4.0
